## WE Simulation of Na - Cl association kinetics

### Analysis of a longer simulation

We have provided you with the log file, `nacl.log.reference` obtained when this simulation was run for 500 cycles (each cycle being 2ps, rather than 1ps as above).

In [ ]:
# Extract data from the log file. Get:
#
# n_walkers: the number of walkers each cycle
# flux: the recycled flux, each cycle
# bin_weights: a dictionary with the cumulative weight of simulation in each bin
#
with open('nacl.log.reference') as f:
    data = f.readlines()

n_walkers = []
flux = []
for d in data[1:-1]:
    w = d.split()
    n_walkers.append(int(w[1]))
    flux.append(float(w[4]))

n_walkers = np.array(n_walkers)
flux = np.array(flux)
bin_weights = eval(data[-1])

# normalise bin weights:
mean_weights = np.array(list(bin_weights.values()))
mean_weights /= mean_weights.sum() 

Plot key data:

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

plt.figure(figsize=(10,8))
plt.subplot(221)
plt.plot(flux)
plt.xlabel('cycle #')
plt.ylabel('flux)')
plt.subplot(222)
plt.plot(n_walkers)
plt.xlabel('cycle #')
plt.ylabel('n_walkers)')
plt.subplot(223)
plt.plot(mean_weights)
plt.xlabel('bin #')
plt.ylabel('relative weight)')
print(f'mean flux = {flux[30:].mean():6.4g}')

The erratic pattern of flux recycling, and the rapid increase and then plateauing in the number of walkers each cycle, are apparent. The majority of the simulation weight remains in the last bin (Na-Cl sepaation > 1.5 nm). To calculate the association rate from the flux, we need to decide on where the boundary between the unassociated and associated states is, and - as this is an association rate constant with units of 1/(time\*concentration) - do a volume correction.

Zooming in a bit on the weights data reveals a 'kink' in the profile that is a fair guide to where the transition state probably is (there is no neeed to be super-exact about this in a case like this). It suggests we can regard the first 10 bins as being on the associated side of the barrier, so the rest count towards the unassociated concentration:

In [ ]:
plt.plot(mean_weights[:18])
plt.xlabel('bin #')
plt.ylabel('relative weight)')

Now the volume correction. The maths below calculates this for a triclinic periodic cell:

In [ ]:
bv = inpcrd.boxVectors
a, b, c = [np.linalg.norm(b) for b in bv] # unit cell vector lengths
unit_vectors = [b / np.linalg.norm(b) for b in bv]
cosalpha = np.dot(unit_vectors[1], unit_vectors[2]) #
cosbeta = np.dot(unit_vectors[0], unit_vectors[2])  # unit cell angles
cosgamma = np.dot(unit_vectors[0], unit_vectors[1]) #
volume = a*b*c*(1 - cosalpha**2 - cosbeta**2 - cosgamma**2) + 2* np.sqrt(np.abs(cosalpha*cosbeta*cosgamma))
print(f'unit cell volume = {volume:6.4g} nm**3')

In [ ]:
boundary_bin = 10 # boundary between what's considered "associated" and "disassociated"
w_u = mean_weights[boundary_bin:].sum() / mean_weights.sum()
print(f'unbound weight = {w_u:6.4g}')

NA = 6.022e+23
nm3_to_dm3 = 1e-24
time_step_to_seconds = 1 / 5e11 # the WE simulations are 2 ps per cycle
concentration = w_u / (volume * NA * nm3_to_dm3)
print(f'concentration of unassociated ion = {concentration:6.4g} M')

k_assoc = flux.mean() / (concentration * time_step_to_seconds)
print(f'Association rate constant = {k_assoc:6.4g} / M.second')

The result is quite close to the diffusion limit for bimolecular asssociation in water (about 7e9 /M.second, see [here](https://en.wikipedia.org/wiki/Diffusion-controlled_reaction).

### Experiments to try:

What happens to the predicted association constant if you decide to move the division between bound and unbound states to a different bin boundary?

You will find a restart file for the "bound" state of the NaCl system in this directory. Try to edit the file `config.yaml` to construct a WE workflow to predict the unbinding rate.